In [21]:
import pandas as pd 
import numpy as np 
import faiss
import os 
from pathlib import Path

from utils.constants import *
from utils.indexing import CalculateResults
from utils.general import test_if_should_load

from calculate_index import create_index

bucket_path = "/home/wo/bn/dvarapandita/test-data/pli/vectors/folder0000/"
sample_file = "abhidhamma-ds-ds2__ds2_1_2_root-pli-ms.p"
lang = "pli"
index_method = "cpu"
alignment_method="local"

In [22]:
import pprint
pp = pprint.PrettyPrinter(indent=4).pprint

In [23]:
index = create_index(bucket_path, index_method)
type(index)

VECTOR SHAPE (14709, 100)
hnsw_add_vertices: adding 14709 elements on top of 0 (preset_levels=0)
  max_level = 3
Adding 1 elements at level 3
Adding 11 elements at level 2
Adding 450 elements at level 1
Adding 14247 elements at level 0
Done in 671.999 ms
##  <class 'faiss.swigfaiss_avx2.IndexHNSWFlat'>  ## 


faiss.swigfaiss_avx2.IndexHNSWFlat

In [24]:
c = CalculateResults(bucket_path, lang, index_method, cindex=index, alignment_method=alignment_method)
type(index)

CalculateResults: Index method:  cpu
CalculateResults: Index after reading:  <class 'faiss.swigfaiss_avx2.IndexHNSWFlat'>


faiss.swigfaiss_avx2.IndexHNSWFlat

In [25]:
query_df = pd.read_pickle(bucket_path + sample_file)
query_df

,segmentnr,original,stemmed,weights,vectors,sumvectors
0,ds2.1.2:0.1,Dhammasaṅgaṇī,▁dhamma,1.0,"[0.49842602, 0.13939862, -0.18911129, 0.539696...","[0.16432658831278485, 0.17364023091892397, -0...."
0,ds2.1.2:0.1,Dhammasaṅgaṇī,saṃ,1.0,"[-0.14652283, 0.16191465, -0.37120858, 0.23266...","[-0.03200280417998632, 0.20194721296429632, -0..."
0,ds2.1.2:0.1,Dhammasaṅgaṇī,gaṇ,1.0,"[0.16878396, 0.4747072, -0.08584538, 0.2211898...","[0.0045525858799616474, 0.22670146115124232, -..."
0,ds2.1.2:0.1,Dhammasaṅgaṇī,ī,1.0,"[-0.055903785, -0.06804281, -0.10822103, 0.200...","[-0.10383008072773617, 0.11347169379393261, -0..."
1,ds2.1.2:0.2,2 Niddesa,▁,1.0,"[0.16431575, -0.028084164, -0.09133771, 0.1609...","[-0.17377622624238329, 0.1949457745999098, -0...."
...,...,...,...,...,...,...
521,ds2.1.2:112.12,Asubhajhānaṁ soḷasakkhattukaṁ.,kkhattu,1.0,"[0.6529658, -0.008482565, 0.37597156, -0.43766...","[0.2645092861105998, 0.22942269189904135, 0.02..."
521,ds2.1.2:112.12,Asubhajhānaṁ soḷasakkhattukaṁ.,kaṃ,1.0,"[0.13053386, 0.6576683, -0.092003666, -0.13549...","[0.07028103712946177, 0.3483753204345703, -0.1..."
522,ds2.1.2:112.13,Rūpāvacarakusalaṁ.,▁rūpāvacar,1.0,"[-0.024896527, -0.08411966, -0.16601908, -0.07...","[0.030112489126622677, 0.14218000570933023, -0..."
522,ds2.1.2:112.13,Rūpāvacarakusalaṁ.,akusal,1.0,"[0.01926053, 0.49371558, -0.26873654, 0.356732...","[0.0851215049624443, 0.3684796740611394, -0.22..."


# 1. calc_results_file

In [26]:
query_vectors = c.prepare_query_vectors(query_df)
pp(query_vectors)

array([[ 0.09730737,  0.10282252, -0.11826219, ...,  0.13194938,
        -0.10922762,  0.05833326],
       [-0.02258465,  0.14251584, -0.13153441, ...,  0.21197923,
        -0.01939952,  0.2175415 ],
       [ 0.00337197,  0.16791111, -0.07211661, ...,  0.15969762,
        -0.00312267,  0.16949555],
       ...,
       [ 0.01129539,  0.05333266, -0.07357609, ..., -0.1317116 ,
         0.12194427,  0.01219688],
       [ 0.03932626,  0.17023814, -0.10453948, ..., -0.11367994,
        -0.00721139, -0.02044988],
       [ 0.11355945,  0.06179992, -0.07402539, ..., -0.00374812,
        -0.04581597,  0.05606649]], dtype=float32)


In [27]:
query_results = index.search(query_vectors, QUERY_DEPTH)
pp(type(query_results))
pp(len(query_results))

scores, positions = query_results[0], query_results[1]
pp((type(scores), type(positions)))
pp((scores.shape, positions.shape))

<class 'tuple'>
2
(<class 'numpy.ndarray'>, <class 'numpy.ndarray'>)
((7425, 20), (7425, 20))


## 1.1. How results are extracted by extract_results

In [28]:
query_position = 0 # =iterator
scores[query_position], positions[query_position]

(array([0.        , 0.        , 0.        , 0.        , 0.31007814,
        0.31007814, 0.31007814, 0.31007814, 0.49309957, 0.49309957,
        0.49309957, 0.49309957, 0.6038114 , 0.6038114 , 0.6038114 ,
        0.6038114 , 0.6038114 , 0.6038114 , 0.6038114 , 0.6038114 ],
       dtype=float32),
 array([    0,  7425,  7676,  8581,     1,  7426,  7677,  8582,     2,
         7427,  7678,  8583,  9523,  9899, 10679, 10952, 11158, 11846,
        12015, 14582]))

```
THRESHOLD = {"skt": 0.03,
              "tib": 0.04,
              "chn": 0.01,#0.01, # 0.01 ist default
              "pli": 0.01,
              "eng": 0.45 }

! clean_results_by_threshold spaws scores and positions !

In [29]:
cleaned_positions, cleaned_scores = c.clean_results_by_threshold(scores[query_position], positions[query_position])
cleaned_positions, cleaned_scores

([0, 7425, 7676, 8581], [0.0, 0.0, 0.0, 0.0])

In [30]:
word_data = c.get_word_data(query_position, cleaned_positions, cleaned_scores)
pp(word_data)

[   ['ds2.1.2:0.2', '2 Niddesa ', 'niddes a ▁citt uppād akaṇḍ', 5, 0.0],
    ['ds2.1.3:0.2', '2 Niddesa ', 'niddes a ▁citt uppād akaṇḍ', 7430, 0.0],
    ['ds2.1.9:0.2', '2 Niddesa ', 'niddes a ▁citt uppād akaṇḍ', 7681, 0.0],
    ['ds2.1.5:0.2', '2 Niddesa ', 'niddes a ▁citt uppād akaṇḍ', 8586, 0.0]]


In [31]:
current_query_data = c.get_current_query_data(query_df, query_position)
pp(current_query_data)

{   'segmentnrs': ['ds2.1.2:0.1', 'ds2.1.2:0.2'],
    'sentence': 'Dhammasaṅgaṇī  Dhammasaṅgaṇī  Dhammasaṅgaṇī  Dhammasaṅgaṇī  2 '
                'Niddesa ',
    'stems': '▁dhamma saṃ gaṇ ī ▁'}


In [32]:
results = {
    "query_position": [],
    "query_segmentnr": [],
    "query_stems": [],
    "query_sentence": [],
    "match_segment": [],
    "match_sentence": [],
    "match_stems": [],
    "match_position": [],
    "match_score": []
}

for entry in word_data:
    results["query_position"].append(query_position)
    results["query_segmentnr"].append(current_query_data["segmentnrs"])
    results["query_stems"].append(current_query_data["stems"])
    results["query_sentence"].append(current_query_data["sentence"])
    results["match_segment"].append(entry[0])
    results["match_sentence"].append(entry[1])
    results["match_stems"].append(entry[2])
    results["match_position"].append(entry[3])
    results["match_score"].append(entry[4])
    
pp(results)

{   'match_position': [5, 7430, 7681, 8586],
    'match_score': [0.0, 0.0, 0.0, 0.0],
    'match_segment': [   'ds2.1.2:0.2',
                         'ds2.1.3:0.2',
                         'ds2.1.9:0.2',
                         'ds2.1.5:0.2'],
    'match_sentence': ['2 Niddesa ', '2 Niddesa ', '2 Niddesa ', '2 Niddesa '],
    'match_stems': [   'niddes a ▁citt uppād akaṇḍ',
                       'niddes a ▁citt uppād akaṇḍ',
                       'niddes a ▁citt uppād akaṇḍ',
                       'niddes a ▁citt uppād akaṇḍ'],
    'query_position': [0, 0, 0, 0],
    'query_segmentnr': [   ['ds2.1.2:0.1', 'ds2.1.2:0.2'],
                           ['ds2.1.2:0.1', 'ds2.1.2:0.2'],
                           ['ds2.1.2:0.1', 'ds2.1.2:0.2'],
                           ['ds2.1.2:0.1', 'ds2.1.2:0.2']],
    'query_sentence': [   'Dhammasaṅgaṇī  Dhammasaṅgaṇī  Dhammasaṅgaṇī  '
                          'Dhammasaṅgaṇī  2 Niddesa ',
                          'Dhammasaṅgaṇī  Dhammasaṅgaṇī  D

In [33]:
results: dict = c.extract_results(query_df, query_results)
result_df = pd.DataFrame(results)
result_df

,query_position,query_segmentnr,query_stems,query_sentence,match_segment,match_sentence,match_stems,match_position,match_score
0,0,"[ds2.1.2:0.1, ds2.1.2:0.2]",▁dhamma saṃ gaṇ ī ▁,Dhammasaṅgaṇī Dhammasaṅgaṇī Dhammasaṅgaṇī D...,ds2.1.2:0.2,2 Niddesa,niddes a ▁citt uppād akaṇḍ,5,0.0
1,0,"[ds2.1.2:0.1, ds2.1.2:0.2]",▁dhamma saṃ gaṇ ī ▁,Dhammasaṅgaṇī Dhammasaṅgaṇī Dhammasaṅgaṇī D...,ds2.1.3:0.2,2 Niddesa,niddes a ▁citt uppād akaṇḍ,7430,0.0
2,0,"[ds2.1.2:0.1, ds2.1.2:0.2]",▁dhamma saṃ gaṇ ī ▁,Dhammasaṅgaṇī Dhammasaṅgaṇī Dhammasaṅgaṇī D...,ds2.1.9:0.2,2 Niddesa,niddes a ▁citt uppād akaṇḍ,7681,0.0
3,0,"[ds2.1.2:0.1, ds2.1.2:0.2]",▁dhamma saṃ gaṇ ī ▁,Dhammasaṅgaṇī Dhammasaṅgaṇī Dhammasaṅgaṇī D...,ds2.1.5:0.2,2 Niddesa,niddes a ▁citt uppād akaṇḍ,8586,0.0
4,1,"[ds2.1.2:0.1, ds2.1.2:0.2]",saṃ gaṇ ī ▁ niddes,Dhammasaṅgaṇī Dhammasaṅgaṇī Dhammasaṅgaṇī 2...,ds2.1.2:0.2,2 Niddesa,a ▁citt uppād akaṇḍ a,6,0.0
...,...,...,...,...,...,...,...,...,...
102318,7422,[ds2.1.2:112.13],▁rūpāvacar akusal aṃ,Rūpāvacarakusalaṁ. Rūpāvacarakusalaṁ. Rūpāva...,ds2.1.3:0.1,Dhammasaṅgaṇī,gaṇ ī ▁ niddes a,7427,0.0
102319,7423,[ds2.1.2:112.13],akusal aṃ,Rūpāvacarakusalaṁ. Rūpāvacarakusalaṁ.,ds2.1.3:0.1,Dhammasaṅgaṇī,ī ▁ niddes a ▁citt,7428,0.0
102320,7423,[ds2.1.2:112.13],akusal aṃ,Rūpāvacarakusalaṁ. Rūpāvacarakusalaṁ.,ds2.1.9:0.1,Dhammasaṅgaṇī,ī ▁ niddes a ▁citt,7679,0.0
102321,7424,[ds2.1.2:112.13],aṃ,Rūpāvacarakusalaṁ.,ds2.1.3:0.2,2 Niddesa,▁ niddes a ▁citt uppād,7429,0.0


## 1.2 Pocessing the matches

```
process_matches(query_df, result_df, self.bucket_path + basename, self.lang, alignment_method=self.alignment_method)
```


In [34]:
import json
from merge_results import get_data_dicts, write_to_gzip, construct_path_json

In [35]:
result_df.replace(np.nan, '', regex=True, inplace=True)
inquiry_segments, \
target_segments, \
position_pairs, \
inquiry_segment_position, \
match_segment_position \
        = get_data_dicts(query_df, result_df)



print(inquiry_segments) # segID -> Original
print(target_segments)  # segID -> Original

print(position_pairs) # query_position vs match_position ========================= LIST!!!

print(inquiry_segment_position) # query_position -> segID
print(match_segment_position)   # match_position -> segID

{'ds2.1.2:0.1': 'Dhammasaṅgaṇī ', 'ds2.1.2:0.2': '2 Niddesa ', 'ds2.1.2:0.3': '2.1 Cittuppādakaṇḍa ', 'ds2.1.2:0.4': '2.1.2. Rūpāvacarakusala ', 'ds2.1.2:1.0': '2.1.2.1. Catukkanaya ', 'ds2.1.2:1.1': 'Katame dhammā kusalā? ', 'ds2.1.2:1.2': 'Yasmiṁ samaye rūpūpapattiyā maggaṁ bhāveti vivicceva kāmehi vivicca akusalehi dhammehi savitakkaṁ savicāraṁ vivekajaṁ pītisukhaṁ paṭhamaṁ jhānaṁ upasampajja viharati pathavīkasiṇaṁ, tasmiṁ samaye phasso hoti …pe… avikkhepo hoti …pe… ', 'ds2.1.2:1.3': 'ime dhammā kusalā. ', 'ds2.1.2:2.1': 'Katame dhammā kusalā? ', 'ds2.1.2:2.2': 'Yasmiṁ samaye rūpūpapattiyā maggaṁ bhāveti vitakkavicārānaṁ vūpasamā ajjhattaṁ sampasādanaṁ cetaso ekodibhāvaṁ avitakkaṁ avicāraṁ samādhijaṁ pītisukhaṁ dutiyaṁ jhānaṁ upasampajja viharati pathavīkasiṇaṁ, tasmiṁ samaye phasso hoti, vedanā hoti, saññā hoti, cetanā hoti, cittaṁ hoti, pīti hoti, sukhaṁ hoti, cittassekaggatā hoti, saddhindriyaṁ hoti, vīriyindriyaṁ hoti, satindriyaṁ hoti, samādhindriyaṁ hoti, paññindriyaṁ hoti, m

# -------------------- HARD ROCK STARTS -----------------------

In [36]:
from utils.merging import   get_pair_clusters, \
                            construct_matches_from_pair_clusters, \
                            create_matches_with_text

from posprocess_matches import filter_matches

### 1.2.1 TODO: get_pair_clusters

In [37]:
pair_clusters = get_pair_clusters(position_pairs, WINDOWSIZE[lang])
pair_clusters

[[(0, 7430),
  (0, 7430),
  (1, 7431),
  (2, 7432),
  (3, 7433),
  (4, 7434),
  (5, 7435),
  (6, 7436),
  (7, 7437),
  (8, 7438)],
 [(0, 7681),
  (0, 7681),
  (1, 7682),
  (2, 7683),
  (3, 7684),
  (4, 7685),
  (5, 7686),
  (6, 7687)],
 [(0, 8586),
  (0, 8586),
  (1, 8587),
  (2, 8588),
  (3, 8589),
  (4, 8590),
  (5, 8591),
  (6, 8592)],
 [(1, 6),
  (1, 6),
  (2, 7),
  (3, 8),
  (4, 9),
  (5, 10),
  (6, 11),
  (7, 12),
  (8, 13),
  (9, 14),
  (10, 15),
  (11, 16),
  (12, 17),
  (13, 18),
  (14, 19),
  (15, 20),
  (16, 21),
  (20, 25),
  (26, 31),
  (36, 41),
  (37, 42),
  (38, 43),
  (39, 44),
  (40, 45),
  (41, 46),
  (42, 47),
  (43, 48),
  (44, 49),
  (45, 50),
  (46, 51),
  (47, 52),
  (48, 53),
  (49, 54),
  (50, 55),
  (51, 56),
  (52, 57)],
 [(15, 916), (15, 916), (16, 917), (23, 924), (25, 926)],
 [(16, 12915), (16, 12915)],
 [(16, 13378), (16, 13378)],
 [(17, 637), (17, 637)],
 [(17, 1478), (17, 1478), (21, 1482), (22, 1483)],
 [(17, 2077), (17, 2077), (23, 2083), (26, 2086)]

In [38]:
matches_segments = construct_matches_from_pair_clusters(pair_clusters, inquiry_segment_position, match_segment_position)
pp(matches_segments)

[   [   ['ds2.1.2:0.1', 'ds2.1.2:0.2', 'ds2.1.2:0.3', 'ds2.1.2:0.4'],
        ['ds2.1.3:0.2', 'ds2.1.3:0.3', 'ds2.1.3:0.4']],
    [   ['ds2.1.2:0.1', 'ds2.1.2:0.2', 'ds2.1.2:0.3'],
        ['ds2.1.9:0.2', 'ds2.1.9:0.3', 'ds2.1.9:0.4']],
    [   ['ds2.1.2:0.1', 'ds2.1.2:0.2', 'ds2.1.2:0.3'],
        ['ds2.1.5:0.2', 'ds2.1.5:0.3', 'ds2.1.5:0.4']],
    [[], []],
    [[], []],
    [['ds2.1.2:1.0', 'ds2.1.2:1.1', 'ds2.1.2:1.2'], ['ds2.1.5:69.2']],
    [['ds2.1.2:1.0', 'ds2.1.2:1.1', 'ds2.1.2:1.2'], ['ds2.1.5:76.2']],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [['ds2.1.2:1.1', 'ds2.1.2:1.2'], ['ds2.1.3:1.2']],
    [['ds2.1.2:1.1', 'ds2.1.2:1.2'], ['ds2.1.5:74.2']],
    [['ds2.1.2:1.1', 'ds2.1.2:1.2'], ['ds2.1.5:84.2']],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[], []],
    [[]

In [39]:
match_results = create_matches_with_text(matches_segments, pair_clusters, inquiry_segments, target_segments, lang, alignment_method=alignment_method)
match_results = filter_matches(match_results)
print("After filtering: ", len(match_results))
pp(match_results[0])

Number of matches:  4010
After filtering:  4009
{   'id': 'ds2.1.2:0',
    'inquiry_pos_beg': 0,
    'inquiry_pos_end': 8,
    'par_length': 35,
    'par_offset_beg': 2,
    'par_offset_end': 7,
    'par_segnr': ['ds2.1.3:0.2', 'ds2.1.3:0.3', 'ds2.1.3:0.4'],
    'par_segtext': [   '2 Niddesa ',
                       '2.1 Cittuppādakaṇḍa ',
                       '2.1.3. Arūpāvacarakusala '],
    'par_string': 'Niddesa  2.1 Cittuppādakaṇḍa  2.1.3',
    'root_length': 34,
    'root_offset_beg': 2,
    'root_offset_end': 7,
    'root_segnr': ['ds2.1.2:0.2', 'ds2.1.2:0.3', 'ds2.1.2:0.4'],
    'root_segtext': [   '2 Niddesa ',
                        '2.1 Cittuppādakaṇḍa ',
                        '2.1.2. Rūpāvacarakusala '],
    'root_string': 'Niddesa  2.1 Cittuppādakaṇḍa  2.1.',
    'score': 0.9714285714285714,
    'src_lang': 'pli',
    'target_pos_beg': 7430,
    'target_pos_end': 7438,
    'tgt_lang': 'pli'}


In [40]:
path_json = bucket_path + Path(sample_file).stem + ".json.gz"
json_str = json.dumps(match_results, indent=4, ensure_ascii=False) + "\n"
write_to_gzip(json_str, path_json)